# Baseline Experiment - Verify Pre-optimized Submission

Using the best available pre-optimized submission from snapshots.

In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely.ops import unary_union
import math

# Load the submission
submission_path = '/home/code/experiments/000_baseline/submission.csv'
df = pd.read_csv(submission_path)
print(f"Submission shape: {df.shape}")
print(df.head(10))

In [ ]:
# Define the Christmas tree polygon
def get_tree_polygon():
    """Return the base Christmas tree polygon vertices."""
    # Tree geometry from description
    trunk_width = 0.15
    trunk_height = 0.2
    base_width = 0.7
    mid_width = 0.4
    top_width = 0.25
    tip_y = 0.8
    
    # Build the tree shape (15 vertices)
    # Starting from bottom-left of trunk, going clockwise
    vertices = [
        (-trunk_width/2, -trunk_height),  # trunk bottom-left
        (-trunk_width/2, 0),               # trunk top-left
        (-base_width/2, 0),                # base tier left
        (-mid_width/2, 0.3),               # mid tier left
        (-top_width/2, 0.5),               # top tier left
        (0, tip_y),                        # tip
        (top_width/2, 0.5),                # top tier right
        (mid_width/2, 0.3),                # mid tier right
        (base_width/2, 0),                 # base tier right
        (trunk_width/2, 0),                # trunk top-right
        (trunk_width/2, -trunk_height),    # trunk bottom-right
    ]
    return vertices

BASE_TREE = get_tree_polygon()
print(f"Base tree has {len(BASE_TREE)} vertices")
print("Vertices:", BASE_TREE)

In [ ]:
def transform_tree(x, y, deg):
    """Transform tree to position (x, y) with rotation deg."""
    rad = math.radians(deg)
    cos_a, sin_a = math.cos(rad), math.sin(rad)
    
    transformed = []
    for vx, vy in BASE_TREE:
        # Rotate then translate
        rx = vx * cos_a - vy * sin_a + x
        ry = vx * sin_a + vy * cos_a + y
        transformed.append((rx, ry))
    return transformed

def parse_value(s):
    """Parse submission value (remove 's' prefix)."""
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

# Parse submission
df['x_val'] = df['x'].apply(parse_value)
df['y_val'] = df['y'].apply(parse_value)
df['deg_val'] = df['deg'].apply(parse_value)

# Extract N from id
df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))
df['tree_idx'] = df['id'].apply(lambda x: int(x.split('_')[1]))

print(f"N values: {df['n'].min()} to {df['n'].max()}")
print(f"Total trees: {len(df)}")
print(f"Expected trees: {sum(range(1, 201))} = {200*201//2}")

In [ ]:
def calculate_score(df):
    """Calculate the competition score."""
    total_score = 0
    scores_by_n = {}
    
    for n in range(1, 201):
        n_df = df[df['n'] == n]
        if len(n_df) != n:
            print(f"Warning: N={n} has {len(n_df)} trees, expected {n}")
            continue
        
        # Get all tree polygons for this N
        all_points = []
        for _, row in n_df.iterrows():
            vertices = transform_tree(row['x_val'], row['y_val'], row['deg_val'])
            all_points.extend(vertices)
        
        # Calculate bounding box
        xs = [p[0] for p in all_points]
        ys = [p[1] for p in all_points]
        width = max(xs) - min(xs)
        height = max(ys) - min(ys)
        side = max(width, height)
        
        # Score contribution
        score_n = (side ** 2) / n
        scores_by_n[n] = {'side': side, 'score': score_n}
        total_score += score_n
    
    return total_score, scores_by_n

score, scores_by_n = calculate_score(df)
print(f"\nTotal Score: {score:.6f}")
print(f"Target Score: 68.890873")
print(f"Gap to target: {score - 68.890873:.6f}")

In [ ]:
# Check for overlaps (sample check for a few N values)
from shapely.geometry import Polygon as ShapelyPolygon

def check_overlaps_for_n(df, n, tolerance=1e-9):
    """Check if any trees overlap for a given N."""
    n_df = df[df['n'] == n]
    polygons = []
    
    for _, row in n_df.iterrows():
        vertices = transform_tree(row['x_val'], row['y_val'], row['deg_val'])
        poly = ShapelyPolygon(vertices)
        polygons.append(poly)
    
    overlaps = []
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]):
                intersection = polygons[i].intersection(polygons[j])
                if intersection.area > tolerance:
                    overlaps.append((i, j, intersection.area))
    
    return overlaps

# Check a sample of N values
print("Checking for overlaps...")
for n in [2, 5, 10, 20, 50, 100, 150, 200]:
    overlaps = check_overlaps_for_n(df, n)
    if overlaps:
        print(f"N={n}: {len(overlaps)} overlaps found!")
        for i, j, area in overlaps[:3]:
            print(f"  Trees {i} and {j}: intersection area = {area:.2e}")
    else:
        print(f"N={n}: No overlaps")

print("\nOverlap check complete.")

In [ ]:
# Full overlap check for all N
print("Full overlap check for all N values...")
all_overlaps = {}
for n in range(1, 201):
    overlaps = check_overlaps_for_n(df, n, tolerance=1e-12)
    if overlaps:
        all_overlaps[n] = overlaps
        print(f"N={n}: {len(overlaps)} overlaps")

if all_overlaps:
    print(f"\nTotal N values with overlaps: {len(all_overlaps)}")
else:
    print("\nNo overlaps found in any N value!")

print(f"\nFinal Score: {score:.6f}")

In [ ]:
# Copy to submission folder
import shutil
shutil.copy(submission_path, '/home/submission/submission.csv')
print("Submission copied to /home/submission/submission.csv")

# Save metrics
import json
metrics = {'cv_score': score}
with open('/home/code/experiments/000_baseline/metrics.json', 'w') as f:
    json.dump(metrics, f)
print(f"Metrics saved: {metrics}")